# MediWise appointment: (HMW5 & Demo for project)

By: Nadya Malekpour


---



---



# 1. Environment Preparation

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install openai

In [ ]:
!pip install tenacity
#To use the @retry decorator with wait_random_exponential and stop_after_attempt functionalities,
#you need to install the tenacity module. Tenacity is a Python library for retrying failed
#operations with a flexible set of retry strategies.

In [ ]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI()

# Function calling:
Function calling allows you to connect models like gpt-4o to external tools and systems. This is useful for many things such as empowering AI assistants with capabilities, or building deep integrations between your applications and the models..

In [ ]:
# Dummy functions to simulate appointment system (see availability and make appointments)
# In production, this could be your backend API or an external API
def get_available_appointments(date):
    """Get available appointments for a given date"""
    appointments = {
        "2024-10-22": ["09:00 AM", "11:00 AM", "2:00 PM"],
        "2024-10-23": ["10:00 AM", "1:00 PM", "3:00 PM"],
        "2024-10-24": ["09:30 AM", "11:30 AM", "4:00 PM"]
    }
    return json.dumps(appointments.get(date, []))

def make_appointment(date, time, patient_name):
    """Make an appointment for a given date and time"""
    appointment_info = {
        "status": "confirmed",
        "date": date,
        "time": time,
        "patient_name": patient_name
    }
    return json.dumps(appointment_info)

In [ ]:
# Define the model and retry decorator
GPT_MODEL = "gpt-4o"
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, temperature=0, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

# Define UDFs (user defined “functions):

In [ ]:
# Define the tools (functions) that the model can use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_available_appointments",
            "description": "Get available appointments for a given date",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "The date for which to check appointments (YYYY-MM-DD)",
                    },
                },
                "required": ["date"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "make_appointment",
            "description": "Make an appointment for a given date and time",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "The date for the appointment (YYYY-MM-DD)",
                    },
                    "time": {
                        "type": "string",
                        "description": "The time for the appointment (HH:MM AM/PM)",
                    },
                    "patient_name": {
                        "type": "string",
                        "description": "The name of the patient",
                    },
                },
                "required": ["date", "time", "patient_name"],
            },
        }
    }
]

available_functions = {
    "get_available_appointments": get_available_appointments,
    "make_appointment": make_appointment,
}

In [ ]:
def process_conversation(user_input, conversation_history):
    messages = conversation_history + [{"role": "user", "content": user_input}]
    response = chat_completion_request(messages, temperature=0, tools=tools, tool_choice="auto")
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    #check if the model wanted to call a function
    if tool_calls:
        #call the function
        messages.append(response_message)
        #send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            if function_name == "get_available_appointments":
                function_response = function_to_call(date=function_args.get("date"))
            elif function_name == "make_appointment":
                function_response = function_to_call(
                    date=function_args.get("date"),
                    time=function_args.get("time"),
                    patient_name=function_args.get("patient_name")
                )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = chat_completion_request(messages, temperature=0, tools=tools, tool_choice="auto")
        return second_response.choices[0].message.content, messages
    else:
        return response_message.content, messages

# Example usage
conversation_history = [
    {"role": "system", "content": "You are MediWise, a helpful healthcare assistant. You can help patients view available appointments and schedule appointments."}
]

while True:
    user_input = input("Patient: ")
    if user_input.lower() == 'exit':
        print("MediWise: Thank you for using our service. Take care!")
        break

    response, conversation_history = process_conversation(user_input, conversation_history)
    print("MediWise:", response)

Patient: hi
MediWise: Hello! How can I assist you today?
Patient: i need to see if there is availability for appointment 
MediWise: Sure, I can help with that. Could you please provide the date for which you want to check the available appointments?
Patient: 10-23
MediWise: Could you please provide the full date in the format YYYY-MM-DD so I can check the availability for you?
Patient: 2024-10-23
MediWise: Here are the available appointment times for October 23, 2024:

- 10:00 AM
- 1:00 PM
- 3:00 PM

If you'd like to book an appointment, please let me know your preferred time and your name!
Patient: 1 pm works for me
MediWise: Great! I can schedule an appointment for you at 1:00 PM on October 23, 2024. Could you please provide your name for the appointment?
Patient: my name is NAdya MX but I changed my mind. make an appointment for 3pm please
MediWise: Your appointment has been successfully scheduled for October 23, 2024, at 3:00 PM under the name Nadya MX. If you need any further assi